In [ ]:
This project is a DATA analysis Project based on SQL 

the data used were downloaded from https://www.kaggle.com/code/dimarudov/data-analysis-using-sql/data 


For this project I used Azure data studio as the basic software.

1st step: import the file from kaggle and create the given database in azure data studio and then import the different tables into the given database using the sql schema of the file.

2nd step: after import we have the following tables

country
league
match
player
player_attribute
team
team_attributes

3 step: we can start browsing our different tables in our database.

In this exercise we would like to check the characteristics of the matches, the teams, the players.
first let's look at the countries for which we have the data with the corresponding leagues
first observation is that data are corrupted specially on Mtach table , we will not be able to evaluate KPI based on goal , shotoon and many other criteia 

Select c.name as country ,l.name as league_name from Country as c
inner join League as l on l.country_id = c.id


let's now check the first KPI  , Team performances over leagues 

first we want to check the related country associated to the league 

Select c.name as country ,l.name as league_name from Country as c
inner join League as l on l.country_id = c.id

 now that we have a list of country , check the Match result involving team performances ( win , lost , drwa and win percentages)


with Match_result AS (select match_api_id,season,home_team.team_long_name AS home_team_name ,away_team.team_long_name AS away_team_name,
match.home_team_api_id,match.away_team_api_id,

CASE
     WHEN home_team_goal > away_team_goal THEN 'W'
     WHEN home_team_goal < away_team_goal THEN 'L'
     ELSE 'D'
END AS home_result ,

CASE
     WHEN away_team_goal > home_team_goal THEN 'W'
     WHEN away_team_goal < home_team_goal THEN 'L'
     ELSE 'D'
END AS away_result 
from match
INNER join team as home_team on home_team.team_api_id = home_team_api_id
INNER join team as away_team on away_team.team_api_id = away_team_api_id)

SELECT 
    team_name,
    SUM(total_wins) AS total_wins,
    SUM(total_matches) AS total_match,
    ROUND(SUM(total_wins) * 100.0 / SUM(total_matches),2) AS win_percentage
FROM (
    SELECT 
        home_team_name AS team_name,
        1 as total_matches,
        CASE 
        WHEN home_result = 'W' THEN 1 ELSE 0 END AS total_wins
    FROM 
        Match_result
    UNION ALL
    SELECT 
        away_team_name AS team_name, 1 as total_matches,
        CASE 
        WHEN away_result = 'W' THEN 1 ELSE 0 END AS total_wins
    FROM 
        match_result
) AS combined_results
GROUP BY 
    team_name
ORDER BY 
    win_percentage DESC;




with Match_result_1 AS (select match_api_id,season,date,home_team.team_long_name AS home_team_name ,away_team.team_long_name AS away_team_name,

CASE
     WHEN home_team_goal > away_team_goal THEN 'W'
     WHEN home_team_goal < away_team_goal THEN 'L'
     ELSE 'D'
END AS home_result ,

CASE
     WHEN away_team_goal > home_team_goal THEN 'W'
     WHEN away_team_goal < home_team_goal THEN 'L'
     ELSE 'D'
END AS away_result 
from match
INNER join team as home_team on home_team.team_api_id = home_team_api_id
INNER join team as away_team on away_team.team_api_id = away_team_api_id)

SELECT 
    team_name,
    SUM(total_lost) AS total_lost
FROM (
    SELECT 
        home_team_name AS team_name,
        CASE WHEN home_result = 'L' THEN 1 ELSE 0 END AS total_lost
    FROM 
        match_result_1
    UNION ALL
    SELECT 
        away_team_name AS team_name,
        CASE WHEN away_result = 'L' THEN 1 ELSE 0 END AS total_lost
    FROM 
        match_result_1
) AS combined_results
GROUP BY 
    team_name
ORDER BY 
    total_lost DESC;




  # Team with highest winning game home 


SELECT top 50 home_team.team_long_name AS home_team_name,
    COUNT(match.match_api_id) AS total_matches,
    SUM(
        CASE
            WHEN match.home_team_goal > match.away_team_goal THEN 1
            ELSE 0
        END
    ) AS total_home_wins 
FROM 
    match
INNER JOIN 
    team AS home_team ON home_team.team_api_id = match.home_team_api_id
GROUP BY  
    home_team.team_long_name
ORDER BY total_home_wins DESC



# team with highest wining game away 


SELECT  away_team.team_long_name AS away_team_name,
    COUNT(match.match_api_id) AS total_matches,
    SUM(
        CASE
            WHEN match.away_team_goal > match.home_team_goal  THEN 1
            ELSE 0
        END
    ) AS total_away_wins 
FROM 
    match
INNER JOIN 
    team AS away_team ON away_team.team_api_id = match.home_team_api_id
GROUP BY  
    away_team.team_long_name
ORDER BY total_away_wins DESC



#Checking player attributes based on different criteria ( shot , head )


Select stage , match_api_id ,home_team_api_id ,away_team_api_id, possession , shoton , shotoff, foulcommit , card ,[cross] , corner , possession from [match] where possession is not NULL;

Select sum (TA.chanceCreationShooting) as Total_shot,t.team_long_name  as Team_name 
from team as t
JOIN Team_Attributes as TA on TA.team_api_id = t.team_api_id
GROUP BY (t.team_long_name )
ORDER BY Total_shot DESC


Select top 50 AVG(heading_accuracy) as top_heading_player,player_name as name
From Player as pl
Inner join playerattribute as pa  on pa.player_api_id = pl.player_api_id
GROUP by player_name
HAVING AVG(heading_accuracy) > 80
order BY top_heading_player DESC

Select top 50 AVG(dribbling) as top_dribbling_player,player_name as name
From Player as pl
Inner join playerattribute as pa  on pa.player_api_id = pl.player_api_id
GROUP by player_name
HAVING AVG(dribbling) > 80
order BY top_dribbling_player DESC

Select top 50 AVG(free_kick_accuracy) as best_freekick_player,player_name as name
From Player as pl
Inner join playerattribute as pa  on pa.player_api_id = pl.player_api_id
GROUP by player_name
HAVING AVG(heading_accuracy) > 80
order BY best_freekick_player DESC